In [1]:
import torch
import adapters

from transformers import AutoModelForCausalLM, AutoConfig
from adapters import LlamaAdapterModel
from transformers import AutoTokenizer
from adapters.models.llama.adapter_model import LlamaAdapterModel

/mnt/shared_home/vdeshpande/miniconda3/envs/peft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda")

In [3]:
#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1b")

#
#config = AutoConfig.from_pretrained("meta-llama/Llama-2-7b-hf")
#config.load_in_8bit = True



#model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", load_in_8bit=True, torch_dtype=torch.float16)
#model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-1b", load_in_8bit=True)
#adapters.init(model)
#model = LlamaAdapterModel(config=config)


In [7]:
model.eval()
for name, par in model.named_parameters():
    par.requires_grad = False

#model.add_adapter("adapter", config="lora", set_active=True)
#model.to(device)

In [8]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 2048)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-15): 16 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear8bitLt(in_features=2048, out_features=6144, bias=True)
          (dense): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear8bitLt(in_features=2048, out_features=8192, bias=True)
          (dense_4h_to_h): Linear8bitLt(in_features=8192, out_feat

In [9]:
toked = tokenizer.encode_plus(
    "Sentence: I feel good. Sentiment of the previous sentence: Positive. Sentence: I feel bad. Sentiment of the previous sentence: ", return_tensors="pt")
input_ids = toked["input_ids"]
attn_mask = toked["attention_mask"]
input_ids = input_ids.to(device)
attn_mask = attn_mask.to(device)
print(input_ids.shape)
print(input_ids.device)

torch.Size([1, 31])
cuda:0


In [10]:
out_ = model.generate(
    inputs=input_ids,
    max_length=100,
    min_length=2,
    num_beams=1
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/mnt/shared_home/vdeshpande/miniconda3/envs/peft/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 31, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [11]:
out_

tensor([[29710,   566,    27,   309,  1928,  1175,    15, 20580,  2092,   273,
           253,  2045,  6197,    27, 29071,    15, 20580,   566,    27,   309,
          1928,  3076,    15, 20580,  2092,   273,   253,  2045,  6197,    27,
           209,   187]], device='cuda:0')

In [12]:
tokenizer.batch_decode(out_)

['Sentence: I feel good. Sentiment of the previous sentence: Positive. Sentence: I feel bad. Sentiment of the previous sentence: \n']